In [202]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import openpyxl
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from openpyxl.drawing.image import Image
import win32com.client as win32
import os
from PIL import ImageGrab
import win32com.client

olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK


In [145]:
print_to_excel = True

In [207]:
data_BRFHRG = pd.read_excel('BRFHRG FX Exposure 29.04.2024.xlsx', sheet_name='Balance Sheet')
data_BRFTO = pd.read_excel('BRFTO FX Exposure Model 27.04.24.xlsx', sheet_name='Balance Sheet')
query = """
SELECT "amount","presentAmountUSD","orgName","companyCode","bscCode",
  CASE WHEN "bscCode" in ('130','135','BSC130','BSC135') THEN 'Trade receivables'
       WHEN "bscCode" in ('160','165','BSC160','BSC165') THEN 'Advances to suppliers'
       WHEN "bscCode" in ('170','175','BSC170','BSC175') THEN 'Capital advances'
       WHEN "bscCode" in ('140','145','BSC140','BSC145') THEN 'Other receivables'
       WHEN "bscCode" in ('330','BSC330') THEN 'Provisions'
       WHEN "bscCode" in ('300','305','BSC300','BSC305') THEN 'Trade payables'
       WHEN "bscCode" in ('310','315','BSC310','BSC315') THEN 'Other payables'
       WHEN "bscCode" in ('360','365','BSC360','BSC365') THEN 'Advances received'
       WHEN "bscCode" in ('350','355','BSC350','BSC355') THEN 'Loans received'
       WHEN "bscCode" in ('210','215','BSC210','BSC215') THEN 'Loans issued'
       WHEN "bscCode" in ('340','BSC340') THEN 'Tax'
       WHEN "bscCode" in ('410','BSC410') THEN 'Pension'
       ELSE 'Unknown'
    END AS "accountPurpose"
FROM xxwh."unifiedARReport"
WHERE "reportDate"=TO_DATE('2024-04-30','YYYY-MM-DD')
AND "companyCode" in ('BRHRG','BRFTO')
"""
data_UnifiedARReport = export_from_WHWEEK(query)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:172: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


In [ ]:
# BD data
data_UnifiedARReport['index'] = data_UnifiedARReport.index
data_UnifiedARReport_pivot = data_UnifiedARReport.pivot_table(index=['companyCode','accountPurpose'],values=['amount','presentAmountUSD','index'], aggfunc={'amount':sum,'presentAmountUSD':sum,'index':list}).rename(columns={'amount':'sum_amount','presentAmountUSD':'sum_presentAmountUSD','index':'list_index'}).reset_index()
data_UnifiedARReport_pivot = concat_columns(data_UnifiedARReport_pivot,['companyCode','accountPurpose'])
# excel data
counter = 0
for excel_data in [data_BRFHRG, data_BRFTO]:
    counter += 1
    excel_data_work = excel_data[['Balance Name','Saldo Final']]
    excel_data_work['index'] = excel_data_work.index
    if counter == 1:
        excel_data_work['sourse'] = 'BRHRG'
        excel_data_work_BRFHRG = excel_data_work
    else:
        excel_data_work['sourse'] = 'BRFTO'
        excel_data_work_BRFTO = excel_data_work
# concat excel data
excel_concat_data = pd.concat([excel_data_work_BRFHRG,excel_data_work_BRFTO])
excel_concat_data = excel_concat_data.dropna()
rename_dict = {'Trade payable':'Trade payables','Trade Receivables':'Trade receivables','Trade payables - credit assignment - International':'Trade payables','Trade payables - credit assignment Domestic':'Trade payables','Advances from customers':'Advances received'}
excel_concat_data['Balance Name'] = excel_concat_data['Balance Name'].replace(rename_dict)
# pivot
excel_concat_pivot_data = excel_concat_data.pivot_table(index=['sourse','Balance Name'],values=['Saldo Final','index'],aggfunc={'Saldo Final':'sum','index':list}).reset_index().rename(columns={'Saldo Final':'sum_Saldo_Final','index':'list_index'})
excel_concat_pivot_data = concat_columns(excel_concat_pivot_data,['sourse','Balance Name'])
# merge
excel_BD_merge_data = excel_concat_pivot_data.merge(data_UnifiedARReport_pivot,how='outer',left_on='concat_columns',right_on='concat_columns').drop(['concat_columns'],axis=1).sort_values('companyCode')

### to excel
if print_to_excel == True:
    Output_file=f'{str(date.today())}_Brazil_UnifiedARReport.xlsx'
    excel_BD_merge_data.to_excel(Output_file, index=False)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15992\874643092.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_UnifiedARReport_pivot = data_UnifiedARReport.pivot_table(index=['companyCode','accountPurpose'],values=['amount','presentAmountUSD','index'], aggfunc={'amount':sum,'presentAmountUSD':sum,'index':list}).rename(columns={'amount':'sum_amount','presentAmountUSD':'sum_presentAmountUSD','index':'list_index'}).reset_index()
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15992\874643092.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data_UnifiedARReport_pivot = data_UnifiedARReport.pivot_table(in

In [226]:
data_riskDebtAccounts = pd.read_excel('riskDebtAccounts_(16).xlsx', header=5)
data_BCS_mapping = pd.read_excel('SAP ERP-SAP BCS mapping.xlsx', sheet_name='Меппинг_НИт', header=3)
# првоерка сочетаемости списка счетов
data_riskDebtAccounts_work = data_riskDebtAccounts[['type','codeIFRS','bscCode','bsxCode']]
counts_BCS_mapping_list = data_BCS_mapping['Счет ЕПС'].values.tolist()
counts_riskDebtAccounts_list = data_riskDebtAccounts_work.codeIFRS.values.tolist()
assert len([x for x in counts_riskDebtAccounts_list if x in counts_BCS_mapping_list])>0
# merge
riskDebtAccounts_BCS_mapping_data_merge = data_BCS_mapping.merge(data_riskDebtAccounts_work,how='inner',left_on='Счет ЕПС',right_on='codeIFRS').reset_index(drop=True)

In [228]:
UnifiedARReport_riskDebtAccounts_BCS_mapping_data_merge = data_UnifiedARReport.merge(riskDebtAccounts_BCS_mapping_data_merge,how='inner',left_on='bscCode',right_on='bscCode').sort_values('bsxCode').drop(['accountPurpose','Основной счет','Счет ЕПС','Флаг технического счета','codeIFRS','bsxCode','bscCode','orgName'],axis=1).rename(columns={'Unnamed: 3':'description','Unnamed: 1':'description_full'})

excel_concat_data['Balance Name'].unique()
UnifiedARReport_riskDebtAccounts_BCS_mapping_data_merge.description_full.unique()
# UnifiedARReport_riskDebtAccounts_BCS_mapping_data_merge.to_excel('merge_data.xlsx', index=False)